In [ ]:
import os
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import requests
from PIL import Image
from io import BytesIO


In [ ]:
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from PIL import Image
import requests
import os
import time
from io import BytesIO
from selenium import webdriver

def setup_driver(allow_popups=False):
    # Créer une instance de ChromeOptions
    chrome_options = Options()

    if not allow_popups:
        # Désactiver les notifications et pop-ups
        chrome_options.add_argument("--disable-notifications")
        chrome_options.add_argument("--disable-popup-blocking")
        chrome_options.add_argument("--disable-infobars")
        chrome_options.add_argument("--disable-extensions")
        chrome_options.add_argument("--disable-gpu")

        # Désactiver les cookies pour éviter les pop-ups liées aux cookies
        chrome_prefs = {"profile.default_content_setting_values.cookies": 2}
        chrome_options.add_experimental_option("prefs", chrome_prefs)

    # Lancer le navigateur avec ces options
    driver = webdriver.Chrome(options=chrome_options)
    return driver

def scrape_images_with_selenium(queries, num_images=50, folder_name="images", keywords=["stop"], mandatory_keywords={}, allow_popups=False, saving_path="Images", show_logs=True, search_engines=["Google", "Bing", "DuckDuckGo"]):
    # Fonction pour afficher les logs seulement si show_logs est True
    def log(message):
        if show_logs:
            print(message)

    # Créer le dossier principal de base (Saving_Path) s'il n'existe pas
    if not os.path.exists(saving_path):
        os.makedirs(saving_path)

    # Créer un sous-dossier pour cette recherche
    target_folder = os.path.join(saving_path, folder_name)
    if not os.path.exists(target_folder):
        os.makedirs(target_folder)

    # Compter combien d'images existent déjà dans le dossier
    existing_images = len([name for name in os.listdir(target_folder) if os.path.isfile(os.path.join(target_folder, name))])

    # Définir les URLs des moteurs de recherche selon les noms fournis
    List_Urls = {
        "Google": "https://www.google.com/search?q=intitle:{query}&tbm=isch",  # Option 1: ajout de 'intitle:'
        "Bing": "https://www.bing.com/images/search?q={query}&form=HDRSC2",
        "DuckDuckGo": "https://duckduckgo.com/?q={query}&t=hb&iar=images&iax=images&ia=images"  # Option 6: ajout de DuckDuckGo
    }

    # Filtrer les moteurs de recherche à utiliser
    List_Urls = {name: url for name, url in List_Urls.items() if name in search_engines}

    image_counter = existing_images  # Garde un compteur global pour éviter de remplacer les images

    driver = setup_driver(allow_popups=allow_popups)  # Utiliser le nouveau driver configuré

    for lang, lang_queries in queries.items():
        mandatory_words = mandatory_keywords.get(lang, [])  # Obtenir les mots obligatoires pour la langue
        for query in lang_queries:
            for Moteur, Url in List_Urls.items():
                log(f"Récupération des images pour la requête: {query} depuis {Moteur}...")
                driver.get(Url.format(query=query))  # Remplacer {query} par la requête spécifique

                # Faire défiler la page pour charger plus d'images (Option 5: augmenter le nombre de défilements)
                for _ in range(10):  # Augmenté à 15 pour charger plus d'images
                    driver.find_element(By.TAG_NAME, "body").send_keys(Keys.END)
                    time.sleep(2)  # Pause pour laisser la page charger les nouvelles images

                # Obtenir le HTML de la page après le défilement
                soup = BeautifulSoup(driver.page_source, 'html.parser')

                # Récupérer toutes les balises <img>
                img_tags = soup.find_all('img')

                # Extraire les URLs des images
                img_urls = []
                for img in img_tags:
                    img_url = img.get('src')
                    alt_text = img.get('alt', '').lower()  # Récupérer le texte alternatif (utile pour la pertinence)
                    title_text = img.get('title', '').lower()  # Récupérer le texte du titre (Option 7)

                    # Filtrer pour s'assurer que les images contiennent les mots-clés obligatoires dans l'URL, alt-text, ou title-text
                    if img_url and img_url.startswith('http'):
                        if (any(keyword.lower() in img_url.lower() or keyword in alt_text or keyword in title_text for keyword in keywords)
                            and all(m_word.lower() in title_text or m_word in alt_text for m_word in mandatory_words)):  # Option 7: mots obligatoires
                            img_urls.append(img_url)

                log(f"URLs récupérées pour la requête '{query}': {len(img_urls)} images")

                # Télécharger et sauvegarder les images jusqu'à atteindre le nombre requis
                downloaded_images = 0
                for img_url in img_urls:
                    if downloaded_images >= num_images:
                        break  # Si on a atteint le nombre d'images requis, on arrête

                    try:
                        # Téléchargement de l'image
                        log(f"Tentative de téléchargement de l'image {downloaded_images + 1}/{num_images}: {img_url}")
                        img_response = requests.get(img_url)
                        img = Image.open(BytesIO(img_response.content))

                        # Déduire le format de l'image
                        img_format = img.format.lower() if img.format else 'jpeg'
                        img_extension = img_format if img_format != 'jpeg' else 'jpg'

                        # Construire le chemin de sauvegarde avec un compteur dynamique global
                        img_path = os.path.join(target_folder, f"image_{image_counter}.{img_extension}")
                        img.save(img_path)

                        log(f"Image {image_counter + 1} sauvegardée sous {img_path}")
                        image_counter += 1  # Incrémenter le compteur d'images global
                        downloaded_images += 1  # Incrémenter le nombre d'images téléchargées

                    except Exception as e:
                        log(f"Erreur lors du téléchargement de l'image {image_counter + 1}: {e}")

    driver.quit()

# # Utilisation de la fonction avec des requêtes organisées par langue
# queries = {
#     "French": [
#         "panneau+stop+images+reelles",
#     ],
#     "English": [ "Stop+sign+real+images"
#     ],
#     "Spanish": [
#         "señal+de+pare+imágenes+reales"
#     ],
#     "German": ["stop+schild+echte+bilder"]
# }
# # Mots obligatoires pour chaque langue
# mandatory_keywords = {
#     "French": ["stop"]
# ,
#     "English": [],
#     "Spanish": [],
#     "German": []
# }

# keywords = ["panneau", "stop", "pare", "schild"]  # Mots-clés pour filtrer les images
# Allow_Popups = False
# Saving_Path = "Images"  # Chemin de base où les dossiers seront créés
# Folder_Name = "Stop_Signs"  # Nom du dossier pour cette recherche
# Show_Logs = True  # Activer ou désactiver les logs
# Search_Engines = ["Google", "Bing", "DuckDuckGo", "Yahoo"]
#                     # Liste des moteurs de recherche à utiliser

# # Appel de la fonction avec l'option de pop-ups désactivée, le dossier spécifié, et les logs activés
# scrape_images_with_selenium(queries, num_images=50, folder_name=Folder_Name, keywords=keywords, mandatory_keywords=mandatory_keywords, allow_popups=Allow_Popups, saving_path=Saving_Path, show_logs=Show_Logs, search_engines=Search_Engines)


In [ ]:
import json
import os

def load_sign_data(json_file):
    with open(json_file, "r", encoding="utf-8") as file:
        sign_data = json.load(file)
    return sign_data

def scrape_sign_images(json_file, num_images=50, allow_popups=False, saving_path="Images", show_logs=True, search_engines=["Google", "Bing", "DuckDuckGo"]):

    # Create Saving Path if it doesn't exist
    if not os.path.exists(saving_path):
        os.makedirs(saving_path)


    # Fonction pour afficher les logs seulement si show_logs est True
    def log(message):
        if show_logs:
            print(message)

    # Charger les données des panneaux depuis le fichier JSON
    sign_data = load_sign_data(json_file)

    # Préparer la structure des requêtes
    queries = {}
    mandatory_keywords = {}

    # Pour chaque catégorie de panneaux, extraire les traductions dans chaque langue
    for category, signs in sign_data.items():
        # Créer le dossier selon la catégorie de panneau
        category_folder = os.path.join(saving_path, category)
        if not os.path.exists(category_folder):
            os.makedirs(category_folder)

        for sign, translations in signs.items():  # Itérer sur les panneaux
            # Créer un sous-dossier pour chaque panneau
            sign_folder = os.path.join(category_folder, sign)  # Créer le dossier du panneau
            if not os.path.exists(sign_folder):
                os.makedirs(sign_folder)

            for lang, translation_data in translations.items():
                # Ajouter les requêtes pour chaque langue
                if lang not in queries:
                    queries[lang] = []
                queries[lang].append(translation_data['Requête'])

                # Ajouter les mots obligatoires (s'il y en a)
                if lang not in mandatory_keywords:
                    mandatory_keywords[lang] = []
                # mandatory_keywords[lang].extend()

            # Utiliser le code de scraping existant pour chaque langue et chaque moteur de recherche
            scrape_images_with_selenium(queries, num_images=num_images, folder_name=sign_folder, keywords=["panneau" , "sign", "senal", "schild"] , mandatory_keywords=mandatory_keywords, allow_popups=allow_popups, saving_path=sign_folder, show_logs=show_logs, search_engines=search_engines)

    log("Scraping terminé et les images sont triées dans les dossiers correspondants.")

# Exemple d'utilisation avec un fichier JSON
json_file = "data.json"  # Remplace par le nom de ton fichier JSON
scrape_sign_images(json_file, num_images=50, saving_path="Images", show_logs=True, search_engines=["Google", "Bing", "DuckDuckGo"])
